In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import shutil
import requests, zipfile, io
import sklearn
import fancyimpute

In [3]:
df = pd.read_csv('Combined.csv', low_memory=False)
df.describe()

,Unnamed: 0,Seq,Dur,RunTime,Mean,Sum,Min,Max,sTos,dTos,...,DstRate,SrcWin,DstWin,sVid,dVid,SrcTCPBase,DstTCPBase,TcpRtt,SynAck,AckDat
count,1.215890e+06,1.215890e+06,1.215890e+06,1.215890e+06,1.215890e+06,1.215890e+06,1.215890e+06,1.215890e+06,1.215676e+06,272823.000000,...,1.215890e+06,2.424200e+05,1.770780e+05,114571.0,2009.0,2.786710e+05,2.300470e+05,1.215890e+06,1.215890e+06,1.215890e+06
mean,3.158885e+05,4.034334e+04,1.364841e+00,1.364841e+00,1.364841e+00,1.364841e+00,1.364841e+00,1.364841e+00,8.643825e-01,2.637593,...,4.999662e+02,9.149689e+05,6.951336e+04,610.0,610.0,2.044248e+09,2.146305e+09,4.669759e-03,5.816065e-04,4.088152e-03
std,1.946853e+05,3.777201e+04,1.691295e+00,1.691295e+00,1.691295e+00,1.691295e+00,1.691295e+00,1.691295e+00,1.251895e+01,21.037454,...,5.657329e+04,4.991476e+06,2.021164e+05,0.0,0.0,1.233429e+09,1.243342e+09,1.742817e-02,1.305568e-02,1.053588e-02
min,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,610.0,610.0,1.230730e+05,3.308900e+04,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.519860e+05,8.861000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,...,0.000000e+00,5.657600e+04,6.476800e+04,610.0,610.0,9.862475e+08,1.070929e+09,0.000000e+00,0.000000e+00,0.000000e+00
50%,3.039720e+05,2.722300e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,...,0.000000e+00,6.259200e+04,6.489600e+04,610.0,610.0,1.994167e+09,2.151747e+09,0.000000e+00,0.000000e+00,0.000000e+00
75%,4.559580e+05,6.331600e+04,2.580399e+00,2.580399e+00,2.580399e+00,2.580399e+00,2.580399e+00,2.580399e+00,0.000000e+00,0.000000,...,0.000000e+00,6.425600e+04,6.502400e+04,610.0,610.0,3.093609e+09,3.224408e+09,0.000000e+00,0.000000e+00,0.000000e+00
max,7.283150e+05,1.372100e+05,1.992063e+01,1.992063e+01,1.992063e+01,1.992063e+01,1.992063e+01,1.992063e+01,2.240000e+02,186.000000,...,2.600000e+07,3.355392e+07,1.677696e+07,610.0,610.0,4.294967e+09,4.294956e+09,1.051236e+00,1.024680e+00,2.667290e-01


In [4]:
# Get duplicated rows
print(df[df.duplicated()])

def drop_duplicated_rows(df):
    df.drop_duplicates(inplace=True)
    return df

df = drop_duplicated_rows(df)
print(df[df.duplicated()])

        Unnamed: 0  Seq  Dur  RunTime  Mean  Sum  Min  Max Proto  sTos  ...  \
728316           0    1  0.0      0.0   0.0  0.0  0.0  0.0  icmp   0.0  ...   

        sVid dVid SrcTCPBase  DstTCPBase  TcpRtt  SynAck  AckDat   Label  \
728316   NaN  NaN        NaN         NaN     0.0     0.0     0.0  Benign   

        Attack Type  Attack Tool  
728316       Benign       Benign  

[1 rows x 52 columns]
Empty DataFrame
Columns: [Unnamed: 0, Seq, Dur, RunTime, Mean, Sum, Min, Max, Proto, sTos, dTos, sDSb, dDSb, sTtl, dTtl, sHops, dHops, Cause, TotPkts, SrcPkts, DstPkts, TotBytes, SrcBytes, DstBytes, Offset, sMeanPktSz, dMeanPktSz, Load, SrcLoad, DstLoad, Loss, SrcLoss, DstLoss, pLoss, SrcGap, DstGap, Rate, SrcRate, DstRate, State, SrcWin, DstWin, sVid, dVid, SrcTCPBase, DstTCPBase, TcpRtt, SynAck, AckDat, Label, Attack Type, Attack Tool]
Index: []

[0 rows x 52 columns]


In [5]:
df.columns[df.isna().any()]


Index(['sTos', 'dTos', 'sDSb', 'dDSb', 'sTtl', 'dTtl', 'sHops', 'dHops',
       'SrcGap', 'DstGap', 'SrcWin', 'DstWin', 'sVid', 'dVid', 'SrcTCPBase',
       'DstTCPBase'],
      dtype='object')

In [8]:
df.groupby(['sDSb', 'Attack Type']).agg(count_num=('Attack Type', 'count')).sort_values('count_num', ascending=False)


count_num
sDSb Attack Type              
cs0  Benign             471062
     UDPFlood           457340
     HTTPFlood          140812
     SlowrateDoS         73124
     TCPConnectScan      20052
     SYNScan             20043
     UDPScan             15906
     SYNFlood             9721
ef   Benign               3538
cs0  ICMPFlood            1155
af11 Benign                789
cs6  Benign                640
cs7  Benign                528
af41 Benign                515
52   Benign                144
af12 Benign                137
cs4  Benign                 93
4    Benign                 74
54   Benign                  1
39   Benign                  1

In [7]:
from fancyimpute import IterativeImputer

# Define a function to impute NaN values with the most frequent value in the group
def impute_most_frequent(group):
    mode_value = group.mode().iloc[0] if not group.mode().empty else np.nan
    return group.fillna(mode_value)

def handle_missing_values(df):
    df.drop_duplicates(inplace=True)
    df.drop(['sVid', 'dVid'], axis=1, inplace=True)
    df.dropna(subset=['sTos', 'sHops', 'sTtl', 'sDSb'], how='any', inplace=True)
    df[['dTtl', 'dDSb', 'dTos', 'dHops', 'SrcGap', 'DstGap']] = df.groupby('Attack Type')[['dTtl', 'dDSb', 'dTos', 'dHops', 'SrcGap', 'DstGap']].transform(impute_most_frequent)
    df['dDSb'] = df['dDSb'].fillna('cs0')
    
    df_subset_copy = df[['dTtl', 'dTos', 'dHops', 'SrcGap', 'DstGap', 'SrcTCPBase', 'DstTCPBase', 'SrcWin', 'DstWin']].copy(deep=True)
    mice_imputer = IterativeImputer()
    df_subset_copy.iloc[:, :] = mice_imputer.fit_transform(df_subset_copy)
    df[['dTtl', 'dTos', 'dHops', 'SrcGap', 'DstGap', 'SrcTCPBase', 'DstTCPBase', 'SrcWin', 'DstWin']] = df_subset_copy
    
handle_missing_values(df)
df.isna().any()

/var/folders/59/h4t1gq5x3fq__3l550cxb8440000gn/T/ipykernel_63135/3716790889.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return group.fillna(mode_value)


Unnamed: 0     False
Seq            False
Dur            False
RunTime        False
Mean           False
Sum            False
Min            False
Max            False
Proto          False
sTos           False
dTos           False
sDSb           False
dDSb           False
sTtl           False
dTtl           False
sHops          False
dHops          False
Cause          False
TotPkts        False
SrcPkts        False
DstPkts        False
TotBytes       False
SrcBytes       False
DstBytes       False
Offset         False
sMeanPktSz     False
dMeanPktSz     False
Load           False
SrcLoad        False
DstLoad        False
Loss           False
SrcLoss        False
DstLoss        False
pLoss          False
SrcGap         False
DstGap         False
Rate           False
SrcRate        False
DstRate        False
State          False
SrcWin         False
DstWin         False
SrcTCPBase     False
DstTCPBase     False
TcpRtt         False
SynAck         False
AckDat         False
Label        

In [9]:
df.columns[df.isna().any()]


Index([], dtype='object')

In [26]:
df.isna().sum()


Unnamed: 0     0
Seq            0
Dur            0
RunTime        0
Mean           0
Sum            0
Min            0
Max            0
Proto          0
sTos           0
dTos           0
sDSb           0
dDSb           0
sTtl           0
dTtl           0
sHops          0
dHops          0
Cause          0
TotPkts        0
SrcPkts        0
DstPkts        0
TotBytes       0
SrcBytes       0
DstBytes       0
Offset         0
sMeanPktSz     0
dMeanPktSz     0
Load           0
SrcLoad        0
DstLoad        0
Loss           0
SrcLoss        0
DstLoss        0
pLoss          0
SrcGap         0
DstGap         0
Rate           0
SrcRate        0
DstRate        0
State          0
SrcWin         0
DstWin         0
SrcTCPBase     0
DstTCPBase     0
TcpRtt         0
SynAck         0
AckDat         0
Label          0
Attack Type    0
Attack Tool    0
dtype: int64

In [27]:
categorical_cols = df.select_dtypes('object').columns
categorical_cols[:-3]

Index(['Proto', 'sDSb', 'dDSb', 'Cause', 'State'], dtype='object')

In [12]:
proto_dummies = pd.get_dummies(df.Proto, dtype=int, prefix='Proto')
proto_dummies


,Proto_icmp,Proto_ipv6-icmp,Proto_sctp,Proto_tcp,Proto_udp
0,1,0,0,0,0
1,1,0,0,0,0
2,0,0,0,0,1
3,0,0,0,0,1
4,0,0,0,0,1
...,...,...,...,...,...
1215885,0,0,1,0,0
1215886,0,0,1,0,0
1215887,0,0,0,1,0
1215888,0,0,1,0,0


In [13]:
sDSb_dummies = pd.get_dummies(df.sDSb, dtype=int, prefix='sDSb')
sDSb_dummies

,sDSb_39,sDSb_4,sDSb_52,sDSb_54,sDSb_af11,sDSb_af12,sDSb_af41,sDSb_cs0,sDSb_cs4,sDSb_cs6,sDSb_cs7,sDSb_ef
0,0,0,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1215885,0,0,0,0,0,0,0,0,0,0,0,1
1215886,0,0,0,0,0,0,0,0,0,0,0,1
1215887,0,0,0,0,0,0,0,1,0,0,0,0
1215888,0,0,0,0,0,0,0,0,0,0,0,1


In [15]:
dDSb_dummies = pd.get_dummies(df.dDSb, dtype=int, prefix='dDSb')
dDSb_dummies

,dDSb_af11,dDSb_af12,dDSb_cs0,dDSb_cs1,dDSb_cs4,dDSb_ef
0,0,0,1,0,0,0
1,0,0,1,0,0,0
2,0,0,1,0,0,0
3,0,0,1,0,0,0
4,0,0,1,0,0,0
...,...,...,...,...,...,...
1215885,0,0,0,0,0,1
1215886,1,0,0,0,0,0
1215887,0,0,1,0,0,0
1215888,1,0,0,0,0,0


In [17]:
Cause_dummies = pd.get_dummies(df.Cause, dtype=int, prefix='Cause')
Cause_dummies

,Cause_Shutdown,Cause_Start,Cause_Status
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0
...,...,...,...
1215885,0,0,1
1215886,0,0,1
1215887,0,1,0
1215888,0,0,1


In [19]:
State_dummies = pd.get_dummies(df.State, dtype=int, prefix='State')
State_dummies

,State_ACC,State_CON,State_ECO,State_FIN,State_INT,State_NRS,State_REQ,State_RSP,State_RST,State_TST,State_URP
0,0,0,1,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1215885,0,1,0,0,0,0,0,0,0,0,0
1215886,0,1,0,0,0,0,0,0,0,0,0
1215887,0,1,0,0,0,0,0,0,0,0,0
1215888,0,1,0,0,0,0,0,0,0,0,0


In [20]:
attType_dummies = pd.get_dummies(df['Attack Type'], dtype=int, prefix='AtkType')
attType_dummies

,AtkType_Benign,AtkType_HTTPFlood,AtkType_ICMPFlood,AtkType_SYNFlood,AtkType_SYNScan,AtkType_SlowrateDoS,AtkType_TCPConnectScan,AtkType_UDPFlood,AtkType_UDPScan
0,1,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
1215885,1,0,0,0,0,0,0,0,0
1215886,1,0,0,0,0,0,0,0,0
1215887,1,0,0,0,0,0,0,0,0
1215888,1,0,0,0,0,0,0,0,0


In [21]:
atkTool_dummies = pd.get_dummies(df['Attack Tool'], dtype=int, prefix='AtkTool')
atkTool_dummies

,AtkTool_Benign,AtkTool_Goldeneye,AtkTool_Hping3,AtkTool_Nmap,AtkTool_Slowloris,AtkTool_Torshammer
0,1,0,0,0,0,0
1,1,0,0,0,0,0
2,1,0,0,0,0,0
3,1,0,0,0,0,0
4,1,0,0,0,0,0
...,...,...,...,...,...,...
1215885,1,0,0,0,0,0
1215886,1,0,0,0,0,0
1215887,1,0,0,0,0,0
1215888,1,0,0,0,0,0


In [22]:
Label_dummies = pd.get_dummies(df.Label, dtype=int, prefix='Label')
Label_dummies

,Label_Benign,Label_Malicious
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
1215885,1,0
1215886,1,0
1215887,1,0
1215888,1,0


In [23]:
merged = pd.concat([df, proto_dummies, sDSb_dummies, dDSb_dummies, Cause_dummies, State_dummies, atkTool_dummies, attType_dummies, Label_dummies], axis='columns')
merged

,Unnamed: 0,Seq,Dur,RunTime,Mean,Sum,Min,Max,Proto,sTos,...,AtkType_HTTPFlood,AtkType_ICMPFlood,AtkType_SYNFlood,AtkType_SYNScan,AtkType_SlowrateDoS,AtkType_TCPConnectScan,AtkType_UDPFlood,AtkType_UDPScan,Label_Benign,Label_Malicious
0,0,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,icmp,0.0,...,0,0,0,0,0,0,0,0,1,0
1,1,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,icmp,0.0,...,0,0,0,0,0,0,0,0,1,0
2,2,3,4.998020,4.998020,4.998020,4.998020,4.998020,4.998020,udp,0.0,...,0,0,0,0,0,0,0,0,1,0
3,3,4,4.998037,4.998037,4.998037,4.998037,4.998037,4.998037,udp,0.0,...,0,0,0,0,0,0,0,0,1,0
4,4,5,4.999453,4.999453,4.999453,4.999453,4.999453,4.999453,udp,0.0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1215885,487569,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,sctp,186.0,...,0,0,0,0,0,0,0,0,1,0
1215886,487570,3,0.235607,0.235607,0.235607,0.235607,0.235607,0.235607,sctp,186.0,...,0,0,0,0,0,0,0,0,1,0
1215887,487571,764,0.099927,0.099927,0.099927,0.099927,0.099927,0.099927,tcp,0.0,...,0,0,0,0,0,0,0,0,1,0
1215888,487572,3,1.307852,1.307852,1.307852,1.307852,1.307852,1.307852,sctp,186.0,...,0,0,0,0,0,0,0,0,1,0


In [25]:
merged.columns.tolist()

['Unnamed: 0',
 'Seq',
 'Dur',
 'RunTime',
 'Mean',
 'Sum',
 'Min',
 'Max',
 'Proto',
 'sTos',
 'dTos',
 'sDSb',
 'dDSb',
 'sTtl',
 'dTtl',
 'sHops',
 'dHops',
 'Cause',
 'TotPkts',
 'SrcPkts',
 'DstPkts',
 'TotBytes',
 'SrcBytes',
 'DstBytes',
 'Offset',
 'sMeanPktSz',
 'dMeanPktSz',
 'Load',
 'SrcLoad',
 'DstLoad',
 'Loss',
 'SrcLoss',
 'DstLoss',
 'pLoss',
 'SrcGap',
 'DstGap',
 'Rate',
 'SrcRate',
 'DstRate',
 'State',
 'SrcWin',
 'DstWin',
 'SrcTCPBase',
 'DstTCPBase',
 'TcpRtt',
 'SynAck',
 'AckDat',
 'Label',
 'Attack Type',
 'Attack Tool',
 'Proto_icmp',
 'Proto_ipv6-icmp',
 'Proto_sctp',
 'Proto_tcp',
 'Proto_udp',
 'sDSb_39',
 'sDSb_4',
 'sDSb_52',
 'sDSb_54',
 'sDSb_af11',
 'sDSb_af12',
 'sDSb_af41',
 'sDSb_cs0',
 'sDSb_cs4',
 'sDSb_cs6',
 'sDSb_cs7',
 'sDSb_ef',
 'dDSb_af11',
 'dDSb_af12',
 'dDSb_cs0',
 'dDSb_cs1',
 'dDSb_cs4',
 'dDSb_ef',
 'Cause_Shutdown',
 'Cause_Start',
 'Cause_Status',
 'State_ACC',
 'State_CON',
 'State_ECO',
 'State_FIN',
 'State_INT',
 'State_NRS',


In [28]:
merged.shape[1]

104

In [29]:
merged.drop(columns=['Proto', 'sDSb', 'dDSb', 'Cause', 'State'], inplace=True)

In [33]:
merged.shape[1]

98

In [34]:
merged.describe()

,Seq,Dur,RunTime,Mean,Sum,Min,Max,sTos,dTos,sTtl,...,AtkType_HTTPFlood,AtkType_ICMPFlood,AtkType_SYNFlood,AtkType_SYNScan,AtkType_SlowrateDoS,AtkType_TCPConnectScan,AtkType_UDPFlood,AtkType_UDPScan,Label_Benign,Label_Malicious
count,1.215675e+06,1.215675e+06,1.215675e+06,1.215675e+06,1.215675e+06,1.215675e+06,1.215675e+06,1.215675e+06,1.215675e+06,1.215675e+06,...,1.215675e+06,1.215675e+06,1.215675e+06,1.215675e+06,1.215675e+06,1.215675e+06,1.215675e+06,1.215675e+06,1.215675e+06,1.215675e+06
mean,4.035017e+04,1.365063e+00,1.365063e+00,1.365063e+00,1.365063e+00,1.365063e+00,1.365063e+00,8.643832e-01,9.489157e-01,8.145449e+01,...,1.158303e-01,9.500895e-04,7.996381e-03,1.648714e-02,6.015094e-02,1.649454e-02,3.762025e-01,1.308409e-02,3.928040e-01,6.071960e-01
std,3.777169e+04,1.691351e+00,1.691351e+00,1.691351e+00,1.691351e+00,1.691351e+00,1.691351e+00,1.251896e+01,1.001610e+01,5.592675e+01,...,3.200214e-01,3.080889e-02,8.906427e-02,1.273394e-01,2.377664e-01,1.273675e-01,4.844320e-01,1.136350e-01,4.883741e-01,4.883741e-01
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.600000e+01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.867000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.300000e+01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.723100e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.300000e+01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
75%,6.332700e+04,2.580405e+00,2.580405e+00,2.580405e+00,2.580405e+00,2.580405e+00,2.580405e+00,0.000000e+00,9.489157e-01,6.300000e+01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
max,1.372100e+05,1.992063e+01,1.992063e+01,1.992063e+01,1.992063e+01,1.992063e+01,1.992063e+01,2.240000e+02,1.860000e+02,2.550000e+02,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [32]:
merged.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
df.